In [1]:
import sys
sys.path.append('../')


In [2]:
import spacy
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

from nlp_helper import read_glove_vecs, convert_to_one_hot

F:\Users\xrd03\Anaconda3\envs\tensorflow-gpu\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
F:\Users\xrd03\Anaconda3\envs\tensorflow-gpu\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.


In [8]:
nlp=spacy.load('en')

F:\Users\xrd03\Anaconda3\envs\tensorflow-gpu\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
F:\Users\xrd03\Anaconda3\envs\tensorflow-gpu\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [3]:
#加载训练数据
pos_datas = np.load("../../data/twitter/tech_not/pos_cleaned.npy")
neg_datas = np.load("../../data/twitter/tech_not/neg_cleaned.npy")

np.random.seed(1)
np.random.shuffle(pos_datas)
np.random.shuffle(neg_datas)

pos_datas_train, pos_datas_test = pos_datas[:700],pos_datas[700:800]
neg_datas_train_10, neg_datas_test_10 = neg_datas[:7000],neg_datas[7000:8000]
neg_datas_train_5, neg_datas_test_5 = neg_datas[13600:17100],neg_datas[17100:20000]
neg_datas_train_1_2 = neg_datas[20000:20350]
neg_datas_train_2, neg_datas_test_2 = neg_datas[12000:13400],neg_datas[13400:13600]

In [4]:
w_to_ix, _, w_to_vec_map  = read_glove_vecs("../../data/embedding/glove.twitter.27B.50d.txt")

In [5]:
# train1_2 = np.concatenate((pos_datas_train,neg_datas_train_2), axis = 0)

# np.random.shuffle(train1_2)
# X_train_1_2 = train1_2[:,1]
# Y_train_1_2 = np.array(list(map(int, train1_2[:,0])))
# train1_5 = np.concatenate((pos_datas_train,neg_datas_train_5), axis = 0)

# np.random.shuffle(train1_5)
# X_train_1_5 = train1_5[:,1]
# Y_train_1_5 = np.array(list(map(int, train1_5[:,0])))

In [5]:
pos_datas_50augs = np.tile(pos_datas, (50,1))


In [6]:
train_auged = np.concatenate((pos_datas_50augs,neg_datas), axis = 0)
train_
np.random.shuffle(train_auged )

X_train, X_test = train_auged[:,1][0:80000], train_auged[:,1][80000:]
Y_train, Y_test = np.array(list(map(int, train_auged[:,0])))[0:80000], np.array(list(map(int, train_auged[:,0])))[80000:]

In [53]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = nlp(X[i].lower())
        
        # Initialize j to 0
        
        if len(sentence_words)> max_len:
            continue
        # Loop over the words of sentence_words
        else:
            j = 0
            for w in sentence_words:

                try:
                    X_indices[i, j] = word_to_index[str(w)]
                    j += 1
                except KeyError:
                    continue
                
        print("已完成:{}/{}".format(i,m),end='\r')
                

    return X_indices

In [50]:
str(nlp('a a a a a a')[0])

'a'

In [10]:
w_to_vec_map["cucumber"].shape[0]

50

In [11]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        if word_to_vec_map[word].shape[0]!=50 :
            print("word:", word)
            continue
        emb_matrix[index] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [12]:
# GRADED FUNCTION: Emojify_V2

def twitter_tech_classify_V1(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape=input_shape, dtype ='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences = True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(2)(X)
    # Add a softmax activation
    X = Activation(activation='softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs =sentence_indices ,outputs=X)
    
    
    return model

In [42]:
maxLen = 80

In [43]:

model = twitter_tech_classify_V1((maxLen,), w_to_vec_map, w_to_ix)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 80)                0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 80, 50)            59675700  
_________________________________________________________________
lstm_13 (LSTM)               (None, 80, 128)           91648     
_________________________________________________________________
dropout_13 (Dropout)         (None, 80, 128)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 258       
__________

In [44]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [54]:
X_train_indices = sentences_to_indices(X_train, w_to_ix, maxLen)


X_train_indices.shape[0]

80000

In [77]:
Y_train_oh = convert_to_one_hot(Y_train, C = 2)

In [78]:
model.fit(X_train_indices, Y_train_oh, epochs = 10, batch_size = 1024, shuffle=True)

Epoch 1/10
80000/80000 [==============================] - ETA: 25s - loss: 0.2232 - acc: 0.92 - ETA: 23s - loss: 0.1886 - acc: 0.94 - ETA: 22s - loss: 0.1719 - acc: 0.94 - ETA: 21s - loss: 0.1618 - acc: 0.94 - ETA: 21s - loss: 0.1632 - acc: 0.94 - ETA: 21s - loss: 0.1574 - acc: 0.94 - ETA: 20s - loss: 0.1503 - acc: 0.95 - ETA: 20s - loss: 0.1511 - acc: 0.95 - ETA: 20s - loss: 0.1477 - acc: 0.95 - ETA: 20s - loss: 0.1485 - acc: 0.95 - ETA: 19s - loss: 0.1511 - acc: 0.95 - ETA: 19s - loss: 0.1485 - acc: 0.95 - ETA: 18s - loss: 0.1465 - acc: 0.95 - ETA: 18s - loss: 0.1467 - acc: 0.95 - ETA: 18s - loss: 0.1465 - acc: 0.95 - ETA: 17s - loss: 0.1457 - acc: 0.95 - ETA: 17s - loss: 0.1452 - acc: 0.95 - ETA: 17s - loss: 0.1446 - acc: 0.95 - ETA: 16s - loss: 0.1421 - acc: 0.95 - ETA: 16s - loss: 0.1396 - acc: 0.95 - ETA: 16s - loss: 0.1388 - acc: 0.95 - ETA: 15s - loss: 0.1395 - acc: 0.95 - ETA: 15s - loss: 0.1392 - acc: 0.95 - ETA: 15s - loss: 0.1382 - acc: 0.95 - ETA: 15s - loss: 0.1366 - acc:

80000/80000 [==============================] - ETA: 21s - loss: 0.0876 - acc: 0.97 - ETA: 21s - loss: 0.0958 - acc: 0.97 - ETA: 20s - loss: 0.0978 - acc: 0.97 - ETA: 20s - loss: 0.0934 - acc: 0.97 - ETA: 20s - loss: 0.0928 - acc: 0.97 - ETA: 20s - loss: 0.0969 - acc: 0.97 - ETA: 20s - loss: 0.0958 - acc: 0.97 - ETA: 20s - loss: 0.0939 - acc: 0.97 - ETA: 19s - loss: 0.0913 - acc: 0.97 - ETA: 19s - loss: 0.0915 - acc: 0.97 - ETA: 19s - loss: 0.0911 - acc: 0.97 - ETA: 19s - loss: 0.0936 - acc: 0.97 - ETA: 18s - loss: 0.0920 - acc: 0.97 - ETA: 18s - loss: 0.0897 - acc: 0.97 - ETA: 18s - loss: 0.0882 - acc: 0.97 - ETA: 17s - loss: 0.0895 - acc: 0.97 - ETA: 17s - loss: 0.0891 - acc: 0.97 - ETA: 17s - loss: 0.0889 - acc: 0.97 - ETA: 16s - loss: 0.0895 - acc: 0.97 - ETA: 16s - loss: 0.0891 - acc: 0.97 - ETA: 16s - loss: 0.0895 - acc: 0.97 - ETA: 15s - loss: 0.0901 - acc: 0.97 - ETA: 15s - loss: 0.0905 - acc: 0.97 - ETA: 15s - loss: 0.0902 - acc: 0.97 - ETA: 15s - loss: 0.0898 - acc: 0.97 - ETA

80000/80000 [==============================] - ETA: 21s - loss: 0.0711 - acc: 0.98 - ETA: 21s - loss: 0.0829 - acc: 0.98 - ETA: 21s - loss: 0.0848 - acc: 0.98 - ETA: 20s - loss: 0.0883 - acc: 0.97 - ETA: 20s - loss: 0.0930 - acc: 0.97 - ETA: 20s - loss: 0.0916 - acc: 0.97 - ETA: 19s - loss: 0.0917 - acc: 0.97 - ETA: 19s - loss: 0.0901 - acc: 0.97 - ETA: 19s - loss: 0.0913 - acc: 0.97 - ETA: 19s - loss: 0.0934 - acc: 0.97 - ETA: 19s - loss: 0.0947 - acc: 0.97 - ETA: 18s - loss: 0.0984 - acc: 0.97 - ETA: 18s - loss: 0.0973 - acc: 0.97 - ETA: 18s - loss: 0.0962 - acc: 0.97 - ETA: 17s - loss: 0.0942 - acc: 0.97 - ETA: 17s - loss: 0.0920 - acc: 0.97 - ETA: 17s - loss: 0.0906 - acc: 0.97 - ETA: 17s - loss: 0.0912 - acc: 0.97 - ETA: 16s - loss: 0.0912 - acc: 0.97 - ETA: 16s - loss: 0.0915 - acc: 0.97 - ETA: 16s - loss: 0.0907 - acc: 0.97 - ETA: 15s - loss: 0.0902 - acc: 0.97 - ETA: 15s - loss: 0.0901 - acc: 0.97 - ETA: 15s - loss: 0.0902 - acc: 0.97 - ETA: 15s - loss: 0.0896 - acc: 0.97 - ETA

80000/80000 [==============================] - ETA: 21s - loss: 0.0876 - acc: 0.98 - ETA: 20s - loss: 0.0885 - acc: 0.98 - ETA: 20s - loss: 0.0836 - acc: 0.98 - ETA: 20s - loss: 0.0820 - acc: 0.97 - ETA: 20s - loss: 0.0843 - acc: 0.97 - ETA: 19s - loss: 0.0831 - acc: 0.98 - ETA: 19s - loss: 0.0785 - acc: 0.98 - ETA: 19s - loss: 0.0765 - acc: 0.98 - ETA: 19s - loss: 0.0796 - acc: 0.98 - ETA: 19s - loss: 0.0800 - acc: 0.98 - ETA: 19s - loss: 0.0813 - acc: 0.98 - ETA: 18s - loss: 0.0800 - acc: 0.98 - ETA: 18s - loss: 0.0803 - acc: 0.98 - ETA: 18s - loss: 0.0790 - acc: 0.98 - ETA: 17s - loss: 0.0772 - acc: 0.98 - ETA: 17s - loss: 0.0765 - acc: 0.98 - ETA: 17s - loss: 0.0759 - acc: 0.98 - ETA: 17s - loss: 0.0753 - acc: 0.98 - ETA: 16s - loss: 0.0753 - acc: 0.98 - ETA: 16s - loss: 0.0746 - acc: 0.98 - ETA: 16s - loss: 0.0738 - acc: 0.98 - ETA: 15s - loss: 0.0731 - acc: 0.98 - ETA: 15s - loss: 0.0723 - acc: 0.98 - ETA: 15s - loss: 0.0723 - acc: 0.98 - ETA: 14s - loss: 0.0719 - acc: 0.98 - ETA

80000/80000 [==============================] - ETA: 22s - loss: 0.0918 - acc: 0.97 - ETA: 21s - loss: 0.0766 - acc: 0.98 - ETA: 21s - loss: 0.0957 - acc: 0.97 - ETA: 21s - loss: 0.0908 - acc: 0.97 - ETA: 20s - loss: 0.0829 - acc: 0.97 - ETA: 20s - loss: 0.0834 - acc: 0.97 - ETA: 20s - loss: 0.0802 - acc: 0.97 - ETA: 19s - loss: 0.0852 - acc: 0.97 - ETA: 19s - loss: 0.0840 - acc: 0.97 - ETA: 19s - loss: 0.0817 - acc: 0.97 - ETA: 19s - loss: 0.0827 - acc: 0.97 - ETA: 19s - loss: 0.0848 - acc: 0.97 - ETA: 18s - loss: 0.0835 - acc: 0.97 - ETA: 18s - loss: 0.0828 - acc: 0.97 - ETA: 18s - loss: 0.0815 - acc: 0.97 - ETA: 17s - loss: 0.0829 - acc: 0.97 - ETA: 17s - loss: 0.0836 - acc: 0.97 - ETA: 17s - loss: 0.0828 - acc: 0.98 - ETA: 16s - loss: 0.0826 - acc: 0.98 - ETA: 16s - loss: 0.0825 - acc: 0.98 - ETA: 16s - loss: 0.0820 - acc: 0.98 - ETA: 15s - loss: 0.0811 - acc: 0.98 - ETA: 15s - loss: 0.0812 - acc: 0.98 - ETA: 15s - loss: 0.0807 - acc: 0.98 - ETA: 15s - loss: 0.0808 - acc: 0.98 - ETA

In [16]:
X_train[0]

"Follow me, I'll follow back!"

In [51]:
X_test

array(['GW: TLDR: Elon Musk gets emotional at Tesla’s shareholder meeting, Microsoft’s underwater data center, Amazon’s $200 Echo Look',
       'Here at the Ketton Sports Bash   Come and see us at the Foxes Den! __ __ ',
       '#pornvideos #xxx #xvideos  #porn #sex   #nsfw #porntube  #xvideo   ?Watch at:  pierced - 7 mins - young tube,young xxx,xxx teens,new young sex,dailyporn,young porn hd videos,mobile young xxx porn,xxx young porn,young porn hq,xxx watch free, porno video...',
       ...,
       'Warframe video up, pretty proud of it but had to delay Wind Waker by 2 days to allow it. Enjoy!',
       'million reasons this company could dominate AI',
       'Area rugby teams win state titles '], dtype='<U330')

In [64]:
X_test_indices = sentences_to_indices(X_test, w_to_ix, maxLen)


In [66]:
Y_test_oh = convert_to_one_hot(Y_test, C = 2)

In [67]:
# np.save("../../data/X_test_indices.npy",X_test_indices)
# np.save("../../data/Y_test_oh.npy",Y_test_oh)

In [83]:

loss,accuracy = model.evaluate(X_test_indices,Y_test_oh, batch_size = 2048)


9150/9150 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - 1s 107us/step


In [84]:
print(loss, accuracy)

0.04932589697870401 0.989945356819799


In [118]:
X =np.array(["airplane"])
X = sentences_to_indices(X, w_to_ix, maxLen)
X


array([[481916.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.]])

In [119]:

model.predict(X)

array([[0.98468554, 0.01531447]], dtype=float32)

In [86]:
model.save_weights("weights.h5")

In [87]:
model.save("model_weights.h5")